In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data: Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentaion : bool 
    params_image_size : list 


@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path
    


In [6]:
from CNNproject.constants import * 
from CNNproject.utils.common import read_yaml, create_directories 
import tensorflow as tf 

In [7]:
class ConfigManager : 
    def __init__ (self, config_filepath = CONFIG_YAML, params_filepath = PARMAS_YAML) :
        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config (self) -> PrepareCallbacksConfig : 
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) 
        create_directories([model_ckpt_dir,
                            Path(config.tensorboard_root_log_dir)])
        
        prepare_call_back_config = PrepareCallbacksConfig( 
            root_dir = Path (config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path (config.checkpoint_model_filepath)
        )
        print("pass")
        return prepare_call_back_config


    def get_training_config(self) -> TrainingConfig : 
        training_config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_directories ([Path(training_config.root_dir)])


        training_config= TrainingConfig( 
        root_dir =  Path(training_config.root_dir),
        trained_model_path = Path(training_config.trained_model_path),
        updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
        training_data =  Path (training_data),
        params_epochs = params.EPOCHS,
        params_batch_size = params.BATCH_SIZE,
        params_is_augmentaion = params.AUGMENTATION,
        params_image_size = params.IMAGE_SIZE,
        )
        print("pass")
        return training_config

In [8]:
import os 
import urllib.request as request 
from zipfile import ZipFile
import tensorflow as tf 
import time

class PrepareCallback:
    def __init__ (self, config: PrepareCallbacksConfig) :
        self.config = config

    
    @property
    def _create_tb_callbacks(self): 
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self): 
        return tf.keras.callbacks.ModelCheckpoint( 
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    

    def get_tb_ckpt_callbacks(self): 
        return (
            [self._create_ckpt_callbacks, 
             self._create_tb_callbacks]
        )

In [9]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time
from CNNproject.utils.common import create_directories
import joblib

In [ ]:
class Training: 
    def __init__(self, config: TrainingConfig) : 
        self.config = config 
        tf.config.run_functions_eagerly(True)
        
    def get_base_model (self) :
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        # General data generator parameters
        datagenerator_kwargs = dict(rescale=1./255)

        dataflow_kwargs = dict(                                                
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Step 1: Create Test Generator (20% of the dataset)
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255, validation_split=0.25
        )
        
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation",  # This will be the test set
            shuffle=False,  
            **dataflow_kwargs
        )

   

        # Step 2: Create Training & Validation Generators (80% training, 20% validation from training)
        train_valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255, validation_split=0.2  # 20% of training data will be used for validation
        )

        self.valid_generator = train_valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation",  # Validation set (from training data)
            shuffle=False,  
            **dataflow_kwargs
        )

        # Step 3: Augment the Training Set
        if self.config.params_is_augmentaion: 
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator( 
                rotation_range=40, 
                horizontal_flip=True, 
                width_shift_range=0.2, 
                height_shift_range=0.2, 
                shear_range=0.2, 
                zoom_range=0.2, 
                rescale=1./255, 
                validation_split=0.2  # Same split as validation
            )
        else:
            train_datagenerator = train_valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="training",  # Training set
            shuffle=True,  
            **dataflow_kwargs
        )




    @staticmethod 
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def train (self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size 
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        print("Training samples:", self.train_generator.samples)
        print("Validation samples:", self.valid_generator.samples)
        print(f"Total test images: {self.test_generator.samples}")

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list

        )

        self.save_model(
            path = self.config.trained_model_path, 
            model =self.model 
        )

    def save_test_data(self, path="artifacts/test_data/test_data.pkl"):
        # Ensure the directory exists
        os.makedirs(os.path.dirname(path), exist_ok=True)

        # Save only the necessary data from the test generator
        test_data = {
            'filenames': self.test_generator.filenames,
            'classes': self.test_generator.classes,
            'samples': self.test_generator.samples,
            'batch_size': self.test_generator.batch_size
            }

        # Save the test data to a file
        with open(path, "wb") as f:
            joblib.dump(test_data, f)


In [11]:
try : 
    config = ConfigManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepareCallbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepareCallbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.save_test_data()
    #training.train(callback_list=callback_list)

except Exception as e : 
    raise e 

[2025-02-10 14:12:23,241: INFO: common: yaml file config\config.yaml loadded successfully]
[2025-02-10 14:12:23,243: INFO: common: yaml file params.yaml loadded successfully]
[2025-02-10 14:12:23,243: INFO: common: Created directory at: artifacts]
[2025-02-10 14:12:23,243: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-02-10 14:12:23,247: INFO: common: Created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
pass
[2025-02-10 14:12:23,251: INFO: common: Created directory at: artifacts\training]
pass
Found 96 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
